In [1]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), r'..\tests')))

In [2]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import re
import inspect
from typing import Any, Literal, Optional

In [3]:

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL, CIRCULAR_REF,
    tbl_pattern, rgn_pattern, flatten_sets
    )

from tests.utilities import TableComparator


In [4]:
from tests.fixtures import static_workbook as base_workbook

In [5]:
wb = inspect.unwrap(base_workbook)()
ws = wb.sheets[1]
tbl = ws.tables[1]
df = tbl.data

In [6]:
fmls = """=+$G$2*F13
=+F14+G14
=+SUM(F15:G15)
=+SUM(F16:G16)
=+H16+H15+H14""".split('\n')
fmls

['=+$G$2*F13',
 '=+F14+G14',
 '=+SUM(F15:G15)',
 '=+SUM(F16:G16)',
 '=+H16+H15+H14']

In [7]:
cells = '''$H$13
$H$14
$H$15
$H$16
$H$17
'''.replace('$', '').splitlines()
cells

['H13', 'H14', 'H15', 'H16', 'H17']

In [8]:
cell_to_delete = 'F'
changes = [f'{cell_to_delete}{i}' for i in range(13, 18)]
changes

['F13', 'F14', 'F15', 'F16', 'F17']

In [9]:
err_ref_cell = "'Hoja de Trabajo'!Z0"

In [10]:
def pass_anchors(coord1, coord2):
    return coord2


In [68]:
def shrink_range(linf, lsup, changes, del_what):
    bflag = del_what == 'col'
    k1, k2, fnc, gnc = (2, 3, ord, chr) if bflag else (3, 2, int, str)
    ucol = cell_pattern.match(lsup).group(k1)
    lx_row = cell_pattern.match(changes[0]).group(k2)
    row_changes = [
        tpl[k1 - 1] for x in changes 
        if (tpl := cell_pattern.match(x).groups()) and tpl[k2 - 1] == lx_row and tpl[k1 - 1] <= ucol
    ]
    if not row_changes:
        return linf, lsup
    min_col = fnc(row_changes[0]) if linf in changes else (fnc(row_changes[0]) - 1)
    n = len(row_changes)
    answ = [linf]
    for x in (linf, lsup)[1 - int(linf in changes):]:
        tpl = list(cell_pattern.match(x).groups())
        tpl[2 - int(bflag)] = gnc(max(min_col, (fnc(tpl[2 - int(bflag)]) - n)))
        sht, col, row = tpl
        x = f"'{sht}'!{col}{row}" if sht else f"{col}{row}"
        answ.append(x)

    return answ[-2:]


In [72]:
del_action = 'col'
pattern = rgn_pattern
def replacement(m):
    if ':' in m[0]:
        m = tbl_pattern.match(m[0])
        prefix = f"'{m[1]}'!" if m[1] else ''
        linf, lsup = map(lambda x: f"{prefix}{x}", m[0].replace('$', '').split(':'))
        if err_ref_cell and all(x in changes for x in (linf, lsup)):
            return err_ref_cell
        linf, lsup = shrink_range(linf, lsup, changes, del_action)
        lsup = lsup.split('!')[-1]
        linf, lsup = map(lambda tpl: pass_anchors(*tpl), zip(m[0].split(':'), (linf, lsup)))
        sub_str = ':'.join([linf, lsup])
        return sub_str
    key = m[0].replace('$', '')
    return (err_ref_cell or pass_anchors(m[0], changes[key])) if key in changes else m[0]


In [32]:
scase = ['ABC', 'EFG', 'CDEF'][0]
changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
changes

['A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17']

In [69]:
shrink_range('F15', 'G15', [f'G{x}' for x in range(13, 18)], 'col')

['F15', 'F15']

In [70]:
def test_del_cases(scase, ncase, del_action):
        if del_action == 'col':
                changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
        else:
                changes = [f'{chr(col)}{row}' for col in range(ord('A'), ord('H') + 1) for row in scase]
        fml = fmls[ncase]
        linf, lsup = rgn_pattern.search(fml)[0].split(':')
        l_answ, r_answ = shrink_range(linf, lsup, changes, del_action)
        return fml, (linf, lsup), (l_answ, r_answ)

In [71]:
scase, ncase, del_action = range(15, 20), 0, 'row'
if del_action == 'col':
    answ = locals().get(f'del_{scase}').splitlines()[ncase]
    left, right = rgn_pattern.search(answ)[0].split(':')
else:
    fmls = '''=+SUM(A13:A17)	=+SUM(B13:B17)	=+SUM(C13:C17)	=+SUM(D13:D17)	=+SUM(E13:E17)	=+SUM(F13:F17)	=+SUM(G13:G17)'''.split('\t')
fml, tpl1, tpl2 = test_del_cases(scase, ncase, del_action)
print(fml, tpl1)
# print(tpl2, (left, right))
print(tpl2, ('left', 'right'))

=+SUM(A13:A17) ('A13', 'A17')
('A13', 'A14') ('left', 'right')


In [58]:
del_action = 'col'

for scase in ['ABC', 'EFG', 'CDEF']:
    for ncase in range(5):
        answ = locals().get(f'del_{scase}').splitlines()[ncase]
        left, right = rgn_pattern.search(answ)[0].split(':')
        fml, (linf, lsup), (l_answ, r_answ) = test_del_cases(scase, ncase, del_action)

        if not all(x == y for x, y in zip((left, right), (l_answ, r_answ))):
            print(f'*** {scase=}, {ncase=}')
            print(f'    {fml=}, {linf=}, {lsup=}, {left=}, {right=}')
            print(f'    ({l_answ=}, {l_answ == left}), ({r_answ=}, {r_answ == right})')

In [37]:
fmls = base.split('\n')
fmls

['=+SUM(A13:G13)',
 '=+SUM(C14:G14)',
 '=+SUM(A15:E15)',
 '=+SUM(A16:D16)',
 '=+SUM(A13:G17)']

In [58]:
df = pd.DataFrame(fmls, columns=['fml'], index=pd.Index(cells, name='cell'))
df

,fml
cell,
H13,=+SUM(A13:G13)
H14,=+SUM(C14:G14)
H15,=+SUM(A15:E15)
H16,=+SUM(A16:D16)
H17,=+SUM(A13:G17)


In [ ]:

df.fml.str.replace(
    pattern,
    replacement,
    regex=True
)            


In [3]:
import collections
list(collections.Counter(['B', 'C', 'B', 'C', 'A']).keys())

['B', 'C', 'A']

In [24]:
base = '''=+SUM(A13:G13)
=+SUM(C14:G14)
=+SUM(A15:E15)
=+SUM(A16:D16)
=+SUM(A13:G17)'''


In [25]:
del_ABC = '''=+SUM(A13:D13)
=+SUM(A14:D14)
=+SUM(A15:B15)
=+SUM(A16:A16)
=+SUM(A13:D17)'''

In [26]:
del_EFG = '''=+SUM(A13:D13)
=+SUM(C14:D14)
=+SUM(A15:D15)
=+SUM(A16:D16)
=+SUM(A13:D17)'''

In [27]:
del_CDEF = '''=+SUM(A13:C13)
=+SUM(C14:C14)
=+SUM(A15:B15)
=+SUM(A16:B16)
=+SUM(A13:C17)'''

In [6]:
a, *b = (1,) + (2,3,4)
a, b

(1, [2, 3, 4])

In [7]:
import threading
import time
from threading import Condition

def get_clipboard_content():
    import tkinter as tk

    root = tk.Tk()
    root.withdraw()
    clp_content = root.clipboard_get()
    root.clipboard_clear()
    return clp_content

class CounterThread(threading.Thread):
    def __init__(self, counter, condition):
        super().__init__()
        self.counter = counter
        self.condition = condition
        self.content = []

    def run(self):
        while True:
            if self.condition.counter > 0:
                # print(f"Counter: {self.condition.counter}")
                time.sleep(2)  # Simulate work
                try:
                    clp_content = get_clipboard_content()
                    self.content.append(clp_content)
                    dot = 'C'
                except Exception as e:
                    # print(self.counter)
                    self.counter -= 1
                    print('.', end='', flush=True)
                    n = 0
                    if self.counter == 0:
                        self.condition.counter = 0
                else:
                    n = 1
                    print('C', end='', flush=True)
                finally:
                    with self.condition:
                        self.condition.counter -= n
                        self.condition.notify()  # Notify the waiting thread
            else:
                break

def wait_for_counter(condition):
    with condition:
        while True:
            if condition.wait_for(lambda: condition.counter == 0):
                print("\nCounter reached zero!")
                break

# Example usage:
condition = Condition()
condition.counter = 3  # Initialize the counter

counter_thread = CounterThread(5, condition)
counter_thread.start()

wait_for_counter(condition)
print(counter_thread.content)

-57
..-58
..-59
..-60
..-61
..
Counter reached zero!
[]


-62
.-63
.-64
.-65
.-66
.-67
.-68
.-69
.-70
.-71
.-72
.-73
.-74
.-75
.-76
.